In [1]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def convert_name_columns_to_str(df,lst_cname_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """
    
    for column in lst_cname_columns: # Перебираем список нужных колонок
        df.loc[:,column] = df.loc[:,column].astype(str)
        # Очищаем колонку от пробельных символов с начала и конца
        df.loc[:,column] = df.loc[:,column].apply(lambda x: x.strip())

In [3]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique_filling_area(x):
    """
    Функция для нахождения площади выдела, возвращает площадь выдела если значение площади одинаково и ноль если значения неравны
    
    """
    # создаем список разделяя по точке с запятой
    temp_lst = x.split(';')
    # Создаем множество оставляя только уникальные значения
    temp_set = set(temp_lst)
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        try:
            return float(temp_lst[0])
        except ValueError:
            return 0

In [4]:

file_filing_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'
file_filing_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
path_to_end_folder_filling_area = 'data/'



In [5]:
"""
Создаем датафреймы
Создаем датафрейм реестра считывая первые 5 колонок и пропуская шапку в 8 строк
"""

first_df = pd.read_excel(file_filing_reestr,skiprows=8,usecols=[0,1,2,3,4,5]) # датафрейм реестра
second_df = pd.read_excel(file_filing_table3,skiprows=6) # датафрейм таблицы 3 в которой нужно заполнить площадь


In [6]:
first_df.head(1)

,1,2,3,4,5,6
0,Ангоянское,Ангоянское,NaN,32,1,13


In [7]:
second_df.head(1)

,1,2,3,4,5,6,7,8,9,10,...,32,33,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,NaN,32,1,13.0,...,NaN,0.0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
second_df.columns

Index([          '1',           '2',           '3',           '4',
                   5,             6,             7,             8,
                   9,            10,            11,            12,
                  13,            14,            15,            16,
                  17,            18,            19,            20,
                  21,            22,            23,            24,
                  25,            26,            27,            28,
                  29,            30,            31,            32,
                  33, 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40'],
      dtype='object')

In [8]:
# Переименовываем колонки в датафрейме реестра
first_df.columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел','Площадь выдела']

In [13]:
# Приводим названия колонок к строковому формату
second_df.columns = list(map(str,second_df.columns))

In [14]:
# Удаляем колонки включающие unnamed
second_df.drop([col for col in second_df.columns if 'Unnamed' in col],axis=1,inplace=True)

In [15]:
second_df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33'],
      dtype='object')

In [226]:
# заполняем урочище
first_df['Урочище'] = first_df['Урочище'].fillna('Название урочища не заполнено')
second_df['Урочище'] = second_df['Урочище'].fillna('Название урочища не заполнено')

In [227]:
# Обрабатываем колонку площади
first_df['Площадь выдела'] = first_df['Площадь выдела'].astype(str)
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x: x.replace('nan','0'))
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x: x.replace(' ','0'))
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x: x.replace(',','.'))
first_df['Площадь выдела'] = first_df['Площадь выдела'].apply(lambda x:x.strip())

In [228]:
# Группируем чтобы получить все возможные значения площади для каждого выдела
group_first_df = first_df.groupby(['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']).agg({'Площадь выдела':combine})

In [229]:
group_first_df['Контроль площади'] = group_first_df['Площадь выдела'].apply(check_unique_filling_area)

In [230]:
out_df=group_first_df.reset_index() # Превращаем индексы в колонки 


In [231]:
first_df.columns

Index(['Лесничество', 'Участковое лесничество', 'Урочище', 'Квартал', 'Выдел',
       'Площадь выдела'],
      dtype='object')

In [232]:
second_df.columns

Index(['Лесничество', 'Участковое лесничество', 'Урочище', 'Квартал', 'Выдел',
       'Площадь выдела'],
      dtype='object')

In [233]:
# Получаем список колонок которые нужно привести к строковому виду
out_df_lst_columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']
second_lst_columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']

# Конвертируем все колонки в строковый формат и очищаем от пробельных символов
# Конвертируем нужные нам колонки в str
convert_name_columns_to_str(out_df,first_lst_columns)
convert_name_columns_to_str(second_df,second_lst_columns)

In [234]:


# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
out_df['ID'] = out_df.loc[:,out_df_lst_columns].sum(axis=1)
second_df['ID'] = second_df.loc[:,second_lst_columns].sum(axis=1)

In [235]:
out_df

,Лесничество,Участковое лесничество,Урочище,Квартал,Выдел,Площадь выдела,Контроль площади,ID
0,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13,13.0,АнгоянскоеАнгоянскоеНазвание урочища не заполн...
1,Ангоянское,Ангоянское,Название урочища не заполнено,32,2,7,7.0,АнгоянскоеАнгоянскоеНазвание урочища не заполн...
2,Ангоянское,Ангоянское,Название урочища не заполнено,32,3,11,11.0,АнгоянскоеАнгоянскоеНазвание урочища не заполн...
3,Ангоянское,Ангоянское,Название урочища не заполнено,32,5,10,10.0,АнгоянскоеАнгоянскоеНазвание урочища не заполн...
4,Ангоянское,Ангоянское,Название урочища не заполнено,32,7,78;78;78,78.0,АнгоянскоеАнгоянскоеНазвание урочища не заполн...
...,...,...,...,...,...,...,...,...
91066,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,11,37,37.0,ХоринскоеХоринское сельскоеТехнический участок...
91067,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,13,25;25,25.0,ХоринскоеХоринское сельскоеТехнический участок...
91068,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,16,5.4,5.4,ХоринскоеХоринское сельскоеТехнический участок...
91069,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,17,6.5,6.5,ХоринскоеХоринское сельскоеТехнический участок...


In [236]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Выделы только в реестре'
wb.create_sheet(title='Выделы только в таблице 3', index=1)
wb.create_sheet(title='Площадь для совпадающих выделов', index=2)

# Проводим слияние
itog_df = pd.merge(out_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)

# Записываем каждый датафрейм в соответсвующий лист
left_df = itog_df[itog_df['_merge'] == 'left_only']
left_df.drop(['_merge','ID','Лесничество_y','Участковое лесничество_y','Урочище_y','Квартал_y','Выдел_y','Площадь выдела_y'], axis=1, inplace=True)
left_df.columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел','Значения площади выдела','Контроль площади']

for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Выделы только в реестре'].append(r)


In [237]:
right_df = itog_df[itog_df['_merge'] == 'right_only']
right_df.drop(['_merge','ID','Лесничество_x','Участковое лесничество_x','Урочище_x','Квартал_x','Выдел_x','Площадь выдела_x','Площадь выдела_y','Контроль площади'], axis=1, inplace=True)
right_df.columns= ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']
# Добавляем временную колонку чтобы по ней провести группировку
right_df['temp'] = '1'
# Группируем чтобы оставить только уникальные выделы


group_right_df = right_df.groupby(['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел']).agg({'temp':combine})
# Извлекаем индексы
group_right_df = group_right_df.reset_index()
# Удаляем колонку по которой проводилась группировка
group_right_df.drop(['temp'],axis=1, inplace=True)


for r in dataframe_to_rows(group_right_df, index=False, header=True):
    wb['Выделы только в таблице 3'].append(r)

In [238]:
both_df = itog_df[itog_df['_merge'] == 'both']
both_df['Площадь выдела']= both_df['Контроль площади']

both_df.drop(['_merge','Площадь выдела_x','ID','Лесничество_y','Участковое лесничество_y','Урочище_y','Квартал_y','Выдел_y','Контроль площади','Площадь выдела_y'], axis=1, inplace=True)
both_df.columns = ['Лесничество','Участковое лесничество','Урочище','Квартал','Выдел','Площадь выдела']

In [239]:
both_df

,Лесничество,Участковое лесничество,Урочище,Квартал,Выдел,Площадь выдела
3834,Байкальское,Голондинское,Название урочища не заполнено,2,2,81.0
3835,Байкальское,Голондинское,Название урочища не заполнено,2,2,81.0
3837,Байкальское,Голондинское,Название урочища не заполнено,3,5,6.3
3838,Байкальское,Голондинское,Название урочища не заполнено,3,5,6.3
3839,Байкальское,Голондинское,Название урочища не заполнено,3,5,6.3
...,...,...,...,...,...,...
21847,Байкальское,Туркинское,Название урочища не заполнено,400,21,11.0
21848,Байкальское,Туркинское,Название урочища не заполнено,400,23,63.0
21849,Байкальское,Туркинское,Название урочища не заполнено,400,23,63.0
21850,Байкальское,Туркинское,Название урочища не заполнено,401,1,114.0


In [240]:
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Площадь для совпадающих выделов'].append(r)

In [241]:
# Сохраняем
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_filling_area}/Площади для таблицы 3 от {current_time}.xlsx')
